In [ ]:
import numpy as np 
import pandas as pd 
import os

target_dirs = []
base_dir = 'dataset/include'
for entry in os.listdir(base_dir):
    first_path = os.path.join(base_dir, entry)
    if os.path.isdir(first_path):
        for sub_entry in os.listdir(first_path):
            second_path = os.path.join(first_path, sub_entry)
            if os.path.isdir(second_path):
                target_dirs.append(second_path)
                print(second_path)

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

In [ ]:
def extract_holistic_landmarks(frame, holistic):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = holistic.process(rgb_frame)

    hand_landmarks = []
    face_landmarks = []
    pose_landmarks = []

    if results.left_hand_landmarks:
        hand_landmarks.extend([(lm.x, lm.y, lm.z) for lm in results.left_hand_landmarks.landmark])
    if results.right_hand_landmarks:
        hand_landmarks.extend([(lm.x, lm.y, lm.z) for lm in results.right_hand_landmarks.landmark])

    if results.face_landmarks:
        face_landmarks.extend([(lm.x, lm.y, lm.z) for lm in results.face_landmarks.landmark])

    if results.pose_landmarks:
        pose_landmarks.extend([(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark])

    all_landmarks = {
        'hand_landmarks': np.array(hand_landmarks) if hand_landmarks else None,
        'face_landmarks': np.array(face_landmarks) if face_landmarks else None,
        'pose_landmarks': np.array(pose_landmarks) if pose_landmarks else None,
    }
    
    return all_landmarks

In [ ]:
def process_videos(root_folder):
    holistic = mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5)

    dataset = []
    labels = []

    sign_folders = os.listdir(root_folder)
    print(sign_folders)
    for sign_folder in sign_folders:
        sign_path = os.path.join(root_folder, sign_folder)
        print(f'Processing subcategory: {sign_folder}')

        if os.path.isdir(sign_path):
            video_files = [filename for filename in os.listdir(sign_path) 
                           if filename.endswith(('.MOV', '.mp4', '.avi', '.mkv', '.wmv', '.flv', '.webm'))]
            
            current_video_count = len(video_files)
            print(f'Number of videos files in {sign_folder}: {current_video_count}')
            
            for video_file in video_files:
                video_path = os.path.join(sign_path, video_file)
                print(f'Processing video: {video_path}')

                # Open video file
                cap = cv2.VideoCapture(video_path)

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Extract landmarks from the original frame
                    landmarks = extract_holistic_landmarks(frame, holistic)
                    if (landmarks['hand_landmarks'] is not None or 
                        landmarks['face_landmarks'] is not None or 
                        landmarks['pose_landmarks'] is not None):
                        combined_landmarks = []
                        if landmarks['hand_landmarks'] is not None:
                            combined_landmarks.extend(landmarks['hand_landmarks'])
                        if landmarks['face_landmarks'] is not None:
                            combined_landmarks.extend(landmarks['face_landmarks'])
                        if landmarks['pose_landmarks'] is not None:
                            combined_landmarks.extend(landmarks['pose_landmarks'])

                        dataset.append(combined_landmarks)
                        labels.append(sign_folder)  

                cap.release()

    # Convert dataset and labels to numpy arrays
    dataset = np.array(dataset, dtype=object)  
    labels = np.array(labels)

    name = os.path.basename(os.path.dirname(root_folder)) 

    save_dir = "data"
    np.savez_compressed(f"{save_dir}/holistic_landmarks_{name}.npz", X=dataset, y=labels)

    # Clean up
    holistic.close()

In [ ]:
for path in target_dirs[]:
    name = os.path.basename(os.path.dirname(path))
    process_videos(path)